# JWebbinar 8: Redshift and Template Fitting

This notebook covers basic examples on how a user can measure the redshift of a source using the visualization tool [Jdaviz](https://jdaviz.readthedocs.io/en/latest/) or programmatically with [Specutils](https://specutils.readthedocs.io/en/latest/).

**Content**
- [Resources and documentation](#resources)
- [Installation](#installation)
- [Imports](#imports)
- [Fetch the example data and the mystery data](#data)
- ["By eye" redshift measurement with Specviz](#byeye)
    - Exercise: measure the redshift on the mystery data
- [Redshift measurement with cross-correlation method](#crosscorr)
    - Get a template and prepare it for use
    - Subtract the continuum from the observed spectrum
    - Exercise: fit the continuum using Specviz
    - Run the cross correlation function
- [Template fitting using a library of spectra](#fitting)
    - Get the library of spectra
    - Run the template_match function

**Author**: Camilla Pacifici (cpacifici@stsci.edu)

<a id='resources'></a>
## Resources and documentation

This notebook uses functionality from [Specutils](https://specutils.readthedocs.io/en/stable/) and [Jdaviz](https://jdaviz.readthedocs.io/en/latest/). Developers at the Space Telescope Science Institute are available to answer questions and resolve problems through the [JWST Help Desk](https://jwsthelp.stsci.edu/). If you wish to send feedback or report problems, you can also submit an issue directly on Github, both for [Specutils](https://github.com/astropy/specutils) and for [Jdaviz](https://github.com/spacetelescope/jdaviz).

<a id='installation'></a>
## Installation

This notebook needs jdaviz only:  
`conda create -n jdaviz python=3.8`  
`conda activate jdaviz`  
from the latest release  
`pip install jdaviz`  
or from git  
`pip install git+https://github.com/spacetelescope/jdaviz.git`

<a id='imports'></a>
## Imports

In [ ]:
# This ensures that our notebook is using the full width of the browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# general os
import os
import zipfile
import urllib.request

# numpy
import numpy as np

# specviz
import jdaviz #this is needed to get the version number later
from jdaviz import SpecViz

# astropy
import astropy #again for the version number
import astropy.units as u
from astropy.io import fits,ascii
from astropy.utils.data import download_file
from astropy.modeling.models import Linear1D, Polynomial1D, Chebyshev1D
from astropy.nddata import StdDevUncertainty
from astropy.visualization import quantity_support

# specutils
import specutils #again for the version number
from specutils import Spectrum1D, SpectralRegion
from specutils.fitting import fit_generic_continuum
from specutils.analysis import correlation
from specutils.analysis import template_comparison
from specutils.manipulation import LinearInterpolatedResampler
from specutils.manipulation import extract_region


# matplotlib (in case we need it)
from matplotlib import pyplot as plt
#customization of matplotlib style
plt.rcParams["figure.figsize"] = (10,5)
params={'legend.fontsize':'18','axes.labelsize':'18',
        'axes.titlesize':'18','xtick.labelsize':'18',
        'ytick.labelsize':'18','lines.linewidth':2,
        'axes.linewidth':2,'animation.html': 'html5',
        'figure.figsize':(8,6)}
plt.rcParams.update(params)
plt.rcParams.update({'figure.max_open_warning': 0})

### Versions:

In [ ]:
print("jdaviz:", jdaviz.__version__)
print("astropy:", astropy.__version__)
print("specutils:", specutils.__version__)

<a id='data'></a>
## Fetch the example data and the mystery data

Here we download a spectrum extracted from a NIRSpec IFU simulation, a mystery dataset you will explore later in an exercise, and a model spectrum we will use as template for the redshift measurement. The template is based on a combination of Simple Stellar Population models including emission lines as done in [Pacifici et al. (2012)](https://ui.adsabs.harvard.edu/abs/2012MNRAS.421.2002P/abstract).

In [ ]:
fn_nirspec = download_file('https://stsci.box.com/shared/static/llbr4w8ucqpz6g81hi41v4tnszwi3bq4.fits', cache=True)
fn_mystery = download_file('https://stsci.box.com/shared/static/h51qakmow49j3isdlxeln4fcxrw7fudc.fits', cache=True)
fn_template = download_file('https://stsci.box.com/shared/static/3rkurzwl0l79j70ddemxafhpln7ljle7.dat', cache=True)

<a id='byeye'></a>
## "By eye" redshift measurement with Specviz

Specviz will allow you to match line wavelengths to the emission lines you see in the spectrum. You will be able to do this using the [redshift slider](https://jdaviz.readthedocs.io/en/latest/specviz/redshift.html) at the top of the interface. But first, let us [open the spectrum in Specviz](https://jdaviz.readthedocs.io/en/latest/specviz/import_data.html).

In [ ]:
# Call the app
viz = SpecViz()
viz.app

In [ ]:
# Load spectrum
viz.load_spectrum(fn_nirspec, data_label="NIRSpec")

Now we need to:
- open the ["line list" plugin](https://jdaviz.readthedocs.io/en/latest/specviz/plugins.html?highlight=plugin#line-lists)
- choose pre-loaded lines or add custom lines (the lines will not show in the viewer because they are plotted at restframe)
- input a guess redshift into the redshift box at the top of the interface
- move the slider to get a better match
- use the [zoom tool](https://jdaviz.readthedocs.io/en/latest/specviz/displaying.html#pan-zoom) to get an even better match

### Exercise:  measure the redshift on the mystery data

You can open a new instance of Specviz and re-do all of the above.

*Hint:*  
*viz2 = SpecViz()*  
*viz2.app*

The mystery dataset is a JWST simulation created with [MIRAGE](https://mirage-data-simulator.readthedocs.io/en/latest/) for the NIRISS wide field slitless mode and run through the [JWST Calibration Pipeline](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline). It is a combination of three filters (F115W, F150W, and F200W).

<a id='crosscorr'></a>
## Redshift measurement with cross-correlation method

It is very common in astronomy to measure a redshift using a cross-correlation algorithm. IRAF uses this methodology in its [xcsao](http://tdc-www.harvard.edu/iraf/rvsao/xcsao/xcsao.html) task. Here, we use the Specutils [template cross-correlation](https://specutils.readthedocs.io/en/stable/analysis.html#template-cross-correlation) function to derive the redshift of our source. There are a couple of things that we need to do before we run the correlation algorithm:
- get a template spectrum for the correlation
- subtract the continuum from both the template and the observed spectrum
- make sure the spectra have some overlap in wavelength

### Get a template and prepare it for use

In [ ]:
# Define unit
spec_unit = u.erg / (u.s * u.cm**2 * u.AA)

# Read spectrum with the ascii function
template = ascii.read(fn_template)
# Create Spectrum1D object
template = Spectrum1D(spectral_axis=template['col1']/1E4*u.um, 
                      flux=template['col2']*spec_unit)


In [ ]:
# Cut to useful range - template and obs MUST overlap, so we go to 1.1um
use_tmp = (template.spectral_axis.value > 0.3) & (template.spectral_axis.value < 1.1)
template_cut = Spectrum1D(spectral_axis=template.spectral_axis[use_tmp], flux=template.flux[use_tmp])


In [ ]:
# Look at spectrum
plt.figure(figsize=[10,6])
plt.plot(template_cut.spectral_axis,template_cut.flux)
plt.xlabel("wavelength ({:latex})".format(template_cut.spectral_axis.unit))
plt.ylabel("flux ({:latex})".format(template_cut.flux.unit))
plt.title("Template")
plt.show()

In [ ]:
# Subtract continuum
mask_temp = ((template_cut.spectral_axis.value > 0.31) & (template_cut.spectral_axis.value < 0.37) |
             (template_cut.spectral_axis.value > 0.40) & (template_cut.spectral_axis.value < 0.47) |
             (template_cut.spectral_axis.value > 0.52) & (template_cut.spectral_axis.value < 0.62) |
             (template_cut.spectral_axis.value > 0.70) & (template_cut.spectral_axis.value < 1.05))

template_forcont = Spectrum1D(spectral_axis=template_cut.spectral_axis[mask_temp], flux=template_cut.flux[mask_temp])

# Use fit_generic_continuum
fit_temp = fit_generic_continuum(template_forcont, model=Polynomial1D(5))
cont_temp = fit_temp(template_cut.spectral_axis)
template_sub = template_cut - cont_temp


In [ ]:
# Print Spectrum1D object
template_sub

In [ ]:
# Look at spectrum
plt.figure(figsize=[10,6])
plt.plot(template_sub.spectral_axis,template_sub.flux)
plt.xlabel("wavelength ({:latex})".format(template_sub.spectral_axis.unit))
plt.ylabel("flux ({:latex})".format(template_sub.flux.unit))
plt.title("Continuum subtracted template")
plt.show()

### Subtract the continuum from the observed spectrum

We can use a different approach and do it with [SpectralRegion](https://specutils.readthedocs.io/en/stable/spectral_regions.html) here. We also need to include an uncertianty to the observed spectrum, if it is not included.

In [ ]:
# Read the spectrum directly as Spectrum1D
spec1d = Spectrum1D.read(fn_nirspec)
# Add the uncertainty
spec1d = Spectrum1D(spectral_axis=spec1d.spectral_axis, flux=spec1d.flux, uncertainty=StdDevUncertainty(0.05*(spec1d.flux)))

print(spec1d)

# Define Spectral Region
region = SpectralRegion(1.3*u.um, 1.6*u.um)
# Extract region
spec1d_cont = extract_region(spec1d, region)
# Run fitting function
fit_obs = fit_generic_continuum(spec1d_cont, model=Linear1D(5))
# Apply to spectral axis
cont_obs = fit_obs(spec1d.spectral_axis)

# Subtract continuum
spec1d_sub = spec1d - cont_obs


In [ ]:
# Look at spectrum
plt.figure(figsize=[10,6])
plt.plot(spec1d_sub.spectral_axis,spec1d_sub.flux)
plt.xlabel("wavelength ({:latex})".format(spec1d_sub.spectral_axis.unit))
plt.ylabel("flux ({:latex})".format(spec1d_sub.flux.unit))
plt.title("Continuum subtracted observed spectrum")
plt.show()

### Exercise: fit the continuum using Specviz

We can find and fit the continuum also in Specviz! You will have to:
- open a new instance of Specviz,
- load the template spectrum,
- [define the regions](https://jdaviz.readthedocs.io/en/latest/specviz/displaying.html#defining-spectral-regions) where there are no emission lines,
- and use the plugin [Model Fitting](https://jdaviz.readthedocs.io/en/latest/specviz/plugins.html#model-fitting) to measure the continuum.

### Run the cross correlation function

In [ ]:
# Call the function
corr, lag = correlation.template_correlate(spec1d_sub, template_sub, lag_units=u.one)

# Plot the correlation
plt.plot(lag, corr)
plt.xlabel("Redshift")
plt.ylabel("Correlation")
plt.show()

In [ ]:
# Redshift based on maximum
index_peak = np.argmax(corr)
z = lag[index_peak]

print("Redshift from peak maximum: ", z)

<a id='fitting'></a>
## Template fitting using a library of spectra

There are two other functions that are suitable for redshift fitting: [template_match](https://specutils.readthedocs.io/en/stable/api/specutils.analysis.template_match.html#specutils.analysis.template_match) and [template_redshift](https://specutils.readthedocs.io/en/stable/api/specutils.analysis.template_redshift.html#specutils.analysis.template_redshift). Here we use template_match which lives within [template_comparison](https://specutils.readthedocs.io/en/stable/analysis.html#template-comparison). We need to specify a library of spectra and the range of potential redshifts. The function computes a chi^2 for each model and returns the best metching model, either at fixed redshift or in a range of redshifts. The time this function takes depends heavily on the size of the library and the length of the redshift array. The library contains 100 model spectral from the same approach as our previous model template.

### Get the library of spectra

In [ ]:
# Download the library and unzip
boxlink = 'https://stsci.box.com/shared/static/1d68n0cg38it7ja3rgyvv8gxjezdqjgh.zip'
boxfile = './templates.zip'
urllib.request.urlretrieve(boxlink, boxfile)

zf = zipfile.ZipFile(boxfile, 'r')
zf.extractall()

templatedir = './templates/'

In [ ]:
# Create the list of Spectrum1Ds for the templates
templatelist = []
#for i in range(1, 100):
for i in range(30,40): # Run on smaller portion of the library
    template_file = "{0}{1:05d}.dat".format(templatedir, i)
    template = ascii.read(template_file)
    temp1d = Spectrum1D(spectral_axis=template['col1']/1.E4*u.um, flux=template['col2']*spec_unit)
    templatelist.append(temp1d)

### Run the template_match function

This takes a little while...it is time for your favorite drink!

In [ ]:
# Define the redshift range
min_redshift = 1.500
max_redshift = 1.502
delta_redshift = 0.0005
z_values = np.arange(min_redshift, max_redshift+delta_redshift, delta_redshift)

# Or use a fixed redshift
zz = 1.501

# Define the resample method
resample_method = "flux_conserving"

# Run the function
tm_results = template_comparison.template_match(observed_spectrum=spec1d, 
                                                spectral_templates=templatelist, 
                                                resample_method=resample_method, 
                                                redshift=zz) # Change the redshift variable for list or fixed

In [ ]:
# Print results
tm_results

In [ ]:
# Load the best matching spectrum in Specviz
viz2 = SpecViz()
viz2.app

In [ ]:
viz2.load_spectrum(spec1d, data_label='Observed spectrum')
viz2.load_spectrum(tm_results[0], data_label='Redshifted best template')